In [ ]:
## law_1.docx, law_2.docx
- pinecone store 저장
    - index name: 임의
- RetrievalQA 구현
    - prompt: rlm/rag-prompt
    - 질문: 전세사기

In [6]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone 
from langchain_pinecone import PineconeVectorStore
from langchain import hub
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


load_dotenv()

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

llm = ChatOpenAI()

prompt = hub.pull('rlm/rag-prompt')

loader1 = Docx2txtLoader('law_1.docx')
loader2 = Docx2txtLoader('law_2.docx')

docs1 = loader1.load()
docs2 = loader2.load()

documents = docs1 + docs2

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = text_splitter.split_documents(documents)

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

database = PineconeVectorStore.from_documents(
    index_name='laws-index',
    embedding=embedding,
    documents=document_list,    
)

In [10]:
database = PineconeVectorStore(
    index_name='laws-index',
    embedding=embedding,
)

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

qa_chain = (
    {
        'context': database.as_retriever() | format_docs,
        'question': RunnablePassthrough(),
    }
    | prompt 
    | llm    
    | StrOutputParser() 
)

qa_chain.invoke('전세사기 피해주택 임대인의 국세는 어떻게 계산하나요? 정확한 수식을 알려주세요')

'전세사기피해주택 임대인의 국세는 해당 주택의 시가표준액과 보유한 모든 주택의 가격 합계액을 고려하여 산정됩니다. 국세는 A × (B / C)의 공식에 따라 계산되며, A는 체납한 국세 금액, B는 해당 주택의 가격, C는 임대인이 보유한 모든 주택의 가격 합계액입니다.여기에 기타 부과세와 국세의 경우에 따라 상속세, 증여세, 종합부동산세 등을 고려하여 산정됩니다.'